In [1]:
include("LiPoSID.jl")
using LinearAlgebra

Anmplitude damping

$\begin{bmatrix}
1 & 0 \\ 0 & \sqrt{1-\gamma}
\end{bmatrix}, \quad \begin{bmatrix}
0 & \sqrt{\gamma} \\ 0 & 0 \end{bmatrix}$

In [14]:
γ = 0.5
E = []
push!(E, [1 0
          0 sqrt(1-γ)])
        
push!(E, [0 sqrt(γ)
          0  0 ])

sum(e'e for e in E)

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

In [15]:
[1 1]' * [1 1]/2

2×2 Matrix{Float64}:
 0.5  0.5
 0.5  0.5

In [16]:
ρᵍ₀ = [1. 0]' * [1 0]
ρᵉ₀ = [0. 1]' * [0 1]
ρˣ₀ = [1 1]' * [1 1]/2
ρʸ₀ = [1 -im]' * [1 -im]/2

2×2 Matrix{ComplexF64}:
 0.5+0.0im  0.0-0.5im
 0.0+0.5im  0.5+0.0im

In [17]:
t_steps = 1
ρᵍ = LiPoSID.timeevolution_kraus(t_steps, ρᵍ₀, E)
ρᵉ = LiPoSID.timeevolution_kraus(t_steps, ρᵉ₀, E)
ρˣ = LiPoSID.timeevolution_kraus(t_steps, ρˣ₀, E)
ρʸ = LiPoSID.timeevolution_kraus(t_steps, ρʸ₀, E)

1-element Vector{Matrix{ComplexF64}}:
 [0.5 + 0.0im 0.0 - 0.5im; 0.0 + 0.5im 0.5 + 0.0im]

In [6]:
ρ₁

UndefVarError: UndefVarError: `ρ₁` not defined

In [24]:
t_steps = 2
ρᵍ = LiPoSID.timeevolution_kraus(t_steps, ρᵍ₀, E)
ρᵉ = LiPoSID.timeevolution_kraus(t_steps, ρᵉ₀, E)
ρˣ = LiPoSID.timeevolution_kraus(t_steps, ρˣ₀, E)
ρʸ = LiPoSID.timeevolution_kraus(t_steps, ρʸ₀, E)

A = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ, ρᵍ, ρˣ, ρʸ])
#A[abs.(A) .< 1e-5] .= 0
A

4×4 Matrix{Float64}:
  0.707107     -2.94232e-16  2.41062e-18  -3.29359e-16
  0.0           0.0          0.0           0.0
 -4.996e-16     1.01798e-16  0.5           0.5
 -4.44089e-16   1.42693e-16  1.38778e-16   1.0

In [8]:
ρˣ[1]

2×2 Matrix{Float64}:
 0.5  0.5
 0.5  0.5

In [31]:
ρ₁ = LiPoSID.timeevolution_kraus(2, ρᵍ₀, E)[end]
ρ₄ = LiPoSID.timeevolution_kraus(2, ρᵉ₀, E)[end]

ρˣᶠ = LiPoSID.timeevolution_kraus(2, ρˣ₀, E)[end]
ρʸᶠ = LiPoSID.timeevolution_kraus(2, ρʸ₀, E)[end]

ρ₃= ρˣᶠ - im*ρʸᶠ - (1-im)*(ρ₁+ρ₄)/2 # with "-" sign χ can be diagonalized ???
ρ₂= ρˣᶠ + im*ρʸᶠ - (1+im)*(ρ₁+ρ₄)/2 # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

σₓ = [ 0  1
       1  0 ]  # X gate

Λ = [ I    σₓ
      σₓ  -I ] / 2

#Ρ = [ρ₁ ρ₃ 
#      ρ₄] 


Ρ = [ρ₁ ρ₂ 
     ρ₃ ρ₄] 
    
χ = Λ*Ρ*Λ


4×4 Matrix{ComplexF64}:
    0.728553+0.0im          …         0.125+0.0im
 1.38778e-17+1.38778e-17im     -1.38778e-17-1.38778e-17im
 1.38778e-17+1.38778e-17im     -1.38778e-17-1.38778e-17im
       0.125+0.0im                0.0214466+0.0im

In [26]:
Ρ

4×4 Matrix{ComplexF64}:
      1.0+0.0im          0.0+0.0im          …     0.707107+0.0im
      0.0+0.0im          0.0+0.0im             5.55112e-17+5.55112e-17im
      0.0+0.0im          0.0+0.0im                     0.0+0.0im
 0.707107+0.0im  5.55112e-17-5.55112e-17im             0.5+0.0im

In [20]:
ρ₃

2×2 Matrix{ComplexF64}:
 0.0+0.0im  0.0+0.0im
 1.0+0.0im  0.0+0.0im

In [27]:
d, U = eigen(χ)

U * diagm(d) * U' ≈ χ

ishermitian(χ)

true

In [28]:
d

4-element Vector{Float64}:
 2.220446049250313e-16
 5.551115123125783e-16
 0.25000000000000017
 0.75

In [29]:
function bloch4(ρ)

    b = convert.(Float64, [ ρ[1,2] + ρ[2,1],
                           (ρ[1,2] - ρ[2,1])*im,
                            ρ[1,1] - ρ[2,2],
                            ρ[1,1] + ρ[2,2]       ])

end

function dm_b4(b) 

    ρ = [ 1+b[3]         b[2]-im*b[3]
          b[2]+im*b[3]   1-b[3]       ]/2

end

function 𝓔(ρ₀, A)
    ρᶠ = dm_b4(A * bloch4(ρ₀))
    return ρᶠ
end

function QPT(A)

    ρᵍ₀ = [1. 0]' * [1 0]
    ρᵉ₀ = [0. 1]' * [0 1]
    ρˣ₀ = [1 1]' * [1 1]/2
    ρʸ₀ = [1 -im]' * [1 -im]/2

    ρ₁ = 𝓔(ρᵍ₀, A)
    ρ₄ = 𝓔(ρᵉ₀, A)
    ρ₂ = 𝓔(ρˣ₀, A) - im*𝓔(ρʸ₀, A) - (1-im)*(ρ₁+ρ₄)/2 # with "-" sign χ can be diagonalized ???
    ρ₃ = 𝓔(ρˣ₀, A) + im*𝓔(ρʸ₀, A) - (1+im)*(ρ₁+ρ₄)/2 

    #ρ₂ = 𝓔(ρˣ₀, Ac) - im*𝓔(ρʸ₀, Ac) - (1-im)*(ρ₁+ρ₄)/2 # with "-" sign χ can be diagonalized ???
    #ρ₃ = 𝓔(ρˣ₀, Ac) + im*𝓔(ρʸ₀, Ac) - (1+im)*(ρ₁+ρ₄)/2 

    σₓ = [ 0  1
           1  0 ]  # X gate

    Λ = [ I    σₓ
          σₓ  -I ] / 2

    #Ρ = [ρ₁ ρ₃ 
    #     ρ₂ ρ₄] 

    Ρ = [ρ₁ ρ₂
         ρ₃ ρ₄] 
     
    χ = Λ*Ρ*Λ

    return χ 
end

χ = QPT(A)
d, U = eigen(χ)

U * diagm(d) * U' ≈ χ

d

4-element Vector{ComplexF64}:
 -0.1035533905932738 - 1.3161564380918607e-17im
 0.24999999999999983 - 5.0304391503364406e-18im
 0.24999999999999994 + 0.0im
  0.6035533905932738 + 1.472255657930146e-17im

In [68]:
Ρ

4×4 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  1.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 1.0+0.0im  0.0+0.0im  0.0+0.0im  1.0+0.0im

In [32]:
function operator_sum(χ)

    @assert ishermitian(χ)

    σ = [ [0 1; 1 0], [0 -im; im 0], [1 0; 0 -1], [1 0; 0 1] ]
    σˣ = σ[1]; σʸ = σ[2]; σᶻ = σ[3]; σᴵ = σ[4]; 

    E₀ = I(2)

    E₁ = σˣ  #  σₓ  or X gate
    E₂ = -im * σʸ
    E₃ = σᶻ

    Ẽ = [E₀, E₁, E₂, E₃] 

    d, U = eigen(χ)
    @assert U * diagm(d) * U' ≈ χ
    
    E = []
    for i in 1:size(U)[2]
        Eᵢ = sqrt(Complex(d[i])) * sum(U[j,i] * Ẽ[j] for j in 1:size(U)[1])
        Eᵢ[abs.(Eᵢ) .< 1e-5] .= 0
        push!(E, Eᵢ)
    end
    return E
end

Eᵗᵒᵐ = operator_sum(χ)

sum(e'e for e in Eᵗᵒᵐ)

2×2 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

In [33]:
Eᵗᵒᵐ

4-element Vector{Any}:
 ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
 ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
 ComplexF64[0.0 + 0.0im 0.7071067811865477 - 3.3306690738754706e-16im; 0.0 + 0.0im 0.0 + 0.0im]
 ComplexF64[0.9999999999999999 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.7071067811865474 + 0.0im]

In [35]:
E[1] - Eᵗᵒᵐ[4] 

2×2 Matrix{ComplexF64}:
 1.11022e-16-0.0im          0.0-0.0im
         0.0-0.0im  2.22045e-16-0.0im

In [36]:
E[2] - Eᵗᵒᵐ[3] 

2×2 Matrix{ComplexF64}:
 0.0-0.0im  -1.11022e-16+3.33067e-16im
 0.0-0.0im           0.0-0.0im

In [47]:
Eᵗᵒᵐ[2]

2×2 Matrix{ComplexF64}:
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im

In [48]:
E[2]

2×2 Matrix{Float64}:
 0.0  0.351364
 0.0  0.0

In [49]:
Eᵗᵒᵐ[3]

2×2 Matrix{ComplexF64}:
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im